In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import time

In [2]:
match_history = pd.read_csv('Match Histories of Challenger Players.csv')
match_history.drop('Unnamed: 0', axis = 1, inplace=True)
match_history.head()

,platformId,gameId,champion,queue,season,timestamp,role,lane,champion_name
0,NA1,3084513429,111,420,13,1562228193006,DUO_SUPPORT,BOTTOM,Nautilus
1,NA1,3084446759,69,420,13,1562226080527,SOLO,MID,Cassiopeia
2,NA1,3084435694,22,420,13,1562223632761,DUO_SUPPORT,NONE,Ashe
3,NA1,3083216368,18,420,13,1562124451801,DUO_CARRY,BOTTOM,Tristana
4,NA1,3083201609,4,420,13,1562122608959,DUO_CARRY,BOTTOM,TwistedFate


# There are not a lot of AD Assassins primarily played in the Mid Lane. Currently there are only 3. Qiyana, Zed, and Talon. I expect these champions to be played with a similar play style. To confirm, I will obtain the match details of all of the matches and run a k-means cluster on the champions in each lane to determine the playstyle

In [8]:
match_history.query("champion_name == 'Zed'").describe()

,gameId,champion,queue,season,timestamp
count,3.310000e+02,331.0,331.000000,331.0,3.310000e+02
mean,3.078346e+09,238.0,421.299094,13.0,1.561709e+12
std,7.420572e+06,0.0,10.111708,0.0,7.216490e+08
min,3.018858e+09,238.0,400.000000,13.0,1.555031e+12
25%,3.074165e+09,238.0,420.000000,13.0,1.561407e+12
50%,3.079467e+09,238.0,420.000000,13.0,1.561850e+12
75%,3.083972e+09,238.0,420.000000,13.0,1.562200e+12
max,3.086976e+09,238.0,460.000000,13.0,1.562456e+12


In [9]:
match_history.query("champion_name == 'Talon'").describe()

,gameId,champion,queue,season,timestamp
count,7.500000e+01,75.0,75.000000,75.0,7.500000e+01
mean,3.077735e+09,91.0,420.133333,13.0,1.561612e+12
std,9.534407e+06,0.0,14.472092,0.0,9.465747e+08
min,3.042855e+09,91.0,400.000000,13.0,1.557888e+12
25%,3.072039e+09,91.0,420.000000,13.0,1.561157e+12
50%,3.081236e+09,91.0,420.000000,13.0,1.561975e+12
75%,3.084626e+09,91.0,420.000000,13.0,1.562240e+12
max,3.086975e+09,91.0,450.000000,13.0,1.562455e+12


In [10]:
match_history.query("champion_name == 'Qiyana'").describe()

,gameId,champion,queue,season,timestamp
count,3.640000e+02,364.0,364.000000,364.0,3.640000e+02
mean,3.081840e+09,246.0,419.340659,13.0,1.562028e+12
std,2.552059e+06,0.0,12.358484,0.0,1.957441e+08
min,3.077888e+09,246.0,400.000000,13.0,1.561753e+12
25%,3.079588e+09,246.0,420.000000,13.0,1.561857e+12
50%,3.081616e+09,246.0,420.000000,13.0,1.562004e+12
75%,3.084149e+09,246.0,430.000000,13.0,1.562210e+12
max,3.086965e+09,246.0,470.000000,13.0,1.562454e+12


## Need to do another API call for the match histories

In [3]:
key = 'RGAPI-10eed924-0936-4bf2-9a6e-4a1395674f18'

In [11]:
match_list = list(match_history.gameId)

In [19]:
matches = []
counter = 0
for match in match_list:
    url = 'https://na1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}'.format(match, key)
    response = requests.get(url)
    matches.append(response.json())
    counter +=1
    if counter == 100:
        print(len(matches),' extracted so far')
        time.sleep(120)
        counter = 0

100  extracted so far
200  extracted so far
300  extracted so far
400  extracted so far
500  extracted so far
600  extracted so far
700  extracted so far
800  extracted so far
900  extracted so far
1000  extracted so far
1100  extracted so far
1200  extracted so far
1300  extracted so far
1400  extracted so far
1500  extracted so far
1600  extracted so far
1700  extracted so far
1800  extracted so far
1900  extracted so far
2000  extracted so far
2100  extracted so far
2200  extracted so far
2300  extracted so far
2400  extracted so far
2500  extracted so far
2600  extracted so far
2700  extracted so far
2800  extracted so far
2900  extracted so far
3000  extracted so far
3100  extracted so far
3200  extracted so far
3300  extracted so far
3400  extracted so far
3500  extracted so far
3600  extracted so far
3700  extracted so far
3800  extracted so far
3900  extracted so far
4000  extracted so far
4100  extracted so far
4200  extracted so far
4300  extracted so far
4400  extracted so f

In [23]:
matches[15617]['participants']

[{'participantId': 1,
  'teamId': 100,
  'championId': 40,
  'spell1Id': 4,
  'spell2Id': 7,
  'stats': {'participantId': 1,
   'win': True,
   'item0': 3174,
   'item1': 3107,
   'item2': 3020,
   'item3': 3089,
   'item4': 3285,
   'item5': 3151,
   'item6': 2052,
   'kills': 15,
   'deaths': 10,
   'assists': 35,
   'largestKillingSpree': 4,
   'largestMultiKill': 4,
   'killingSprees': 3,
   'longestTimeSpentLiving': 237,
   'doubleKills': 1,
   'tripleKills': 1,
   'quadraKills': 1,
   'pentaKills': 0,
   'unrealKills': 0,
   'totalDamageDealt': 104370,
   'magicDamageDealt': 100649,
   'physicalDamageDealt': 1876,
   'trueDamageDealt': 1844,
   'largestCriticalStrike': 0,
   'totalDamageDealtToChampions': 31860,
   'magicDamageDealtToChampions': 30845,
   'physicalDamageDealtToChampions': 670,
   'trueDamageDealtToChampions': 344,
   'totalHeal': 36140,
   'totalUnitsHealed': 32,
   'damageSelfMitigated': 28211,
   'damageDealtToObjectives': 2756,
   'damageDealtToTurrets': 2756,

In [2]:
import pickle

In [24]:


output = open('mylist_of_dicts.pkl', 'wb')
pickle.dump(matches,output)
output.close()

In [3]:

pkl_file = open('mylist_of_dicts.pkl', 'rb')

mydict = pickle.load(pkl_file)

pkl_file.close()

mydict[0]['teams']

[{'teamId': 100,
  'win': 'Fail',
  'firstBlood': False,
  'firstTower': False,
  'firstInhibitor': False,
  'firstBaron': False,
  'firstDragon': False,
  'firstRiftHerald': False,
  'towerKills': 1,
  'inhibitorKills': 0,
  'baronKills': 0,
  'dragonKills': 0,
  'vilemawKills': 0,
  'riftHeraldKills': 0,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 120, 'pickTurn': 1},
   {'championId': 43, 'pickTurn': 2},
   {'championId': 91, 'pickTurn': 3},
   {'championId': 84, 'pickTurn': 4},
   {'championId': 99, 'pickTurn': 5}]},
 {'teamId': 200,
  'win': 'Win',
  'firstBlood': True,
  'firstTower': True,
  'firstInhibitor': True,
  'firstBaron': True,
  'firstDragon': True,
  'firstRiftHerald': True,
  'towerKills': 9,
  'inhibitorKills': 1,
  'baronKills': 1,
  'dragonKills': 2,
  'vilemawKills': 0,
  'riftHeraldKills': 1,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 2, 'pickTurn': 6},
   {'championId': 122, 'pickTurn': 7},
   {'championId': 23, 'pickTurn': 8},
   {'champ

In [5]:
mydict[5666]['participants'][4]['stats']

{'participantId': 5,
 'win': False,
 'item0': 1401,
 'item1': 2055,
 'item2': 3065,
 'item3': 3111,
 'item4': 3742,
 'item5': 3109,
 'item6': 3340,
 'kills': 1,
 'deaths': 2,
 'assists': 11,
 'largestKillingSpree': 0,
 'largestMultiKill': 1,
 'killingSprees': 0,
 'longestTimeSpentLiving': 1987,
 'doubleKills': 0,
 'tripleKills': 0,
 'quadraKills': 0,
 'pentaKills': 0,
 'unrealKills': 0,
 'totalDamageDealt': 186479,
 'magicDamageDealt': 84867,
 'physicalDamageDealt': 35974,
 'trueDamageDealt': 65636,
 'largestCriticalStrike': 0,
 'totalDamageDealtToChampions': 10384,
 'magicDamageDealtToChampions': 9226,
 'physicalDamageDealtToChampions': 819,
 'trueDamageDealtToChampions': 338,
 'totalHeal': 29296,
 'totalUnitsHealed': 5,
 'damageSelfMitigated': 50409,
 'damageDealtToObjectives': 33413,
 'damageDealtToTurrets': 650,
 'visionScore': 50,
 'timeCCingOthers': 42,
 'totalDamageTaken': 46647,
 'magicalDamageTaken': 13889,
 'physicalDamageTaken': 30694,
 'trueDamageTaken': 2063,
 'goldEarned'

In [45]:
gameId = []
duration = []

team1_barons = []
team1_dragons = []
team1_ban1 = []
team1_ban2 = []
team1_ban3 = []
team1_ban4 = []
team1_ban5 = []

team1_player1_champion = []
team1_player1_item0 = []
team1_player1_item1 = []
team1_player1_item2 = []
team1_player1_item3 = []
team1_player1_item4 = []
team1_player1_item5 = []
team1_player1_item6 = []
team1_player1_kills = []
team1_player1_deaths = []
team1_player1_assists = []
team1_player1_killing_spree = []
team1_player1_multikills = []
team1_player1_total_damage = []
team1_player1_damage_to_objectives = []
team1_player1_damage_to_turrets = []
team1_player1_damage_taken = []
team1_player1_gold_earned = []
team1_player1_champ_level = []
team1_player1_wards_bought = []
team1_player1_vision_score = []
team1_player1_crowd_control = []
team1_player1_role = []
team1_player1_lane = []

team1_player2_champion = []
team1_player2_item0 = []
team1_player2_item1 = []
team1_player2_item2 = []
team1_player2_item3 = []
team1_player2_item4 = []
team1_player2_item5 = []
team1_player2_item6 = []
team1_player2_kills = []
team1_player2_deaths = []
team1_player2_assists = []
team1_player2_killing_spree = []
team1_player2_multikills = []
team1_player2_total_damage = []
team1_player2_damage_to_objectives = []
team1_player2_damage_to_turrets = []
team1_player2_damage_taken = []
team1_player2_gold_earned = []
team1_player2_champ_level = []
team1_player2_wards_bought = []
team1_player2_vision_score = []
team1_player2_crowd_control = []
team1_player2_role = []
team1_player2_lane = []

team1_player3_champion = []
team1_player3_item0 = []
team1_player3_item1 = []
team1_player3_item2 = []
team1_player3_item3 = []
team1_player3_item4 = []
team1_player3_item5 = []
team1_player3_item6 = []
team1_player3_kills = []
team1_player3_deaths = []
team1_player3_assists = []
team1_player3_killing_spree = []
team1_player3_multikills = []
team1_player3_total_damage = []
team1_player3_damage_to_objectives = []
team1_player3_damage_to_turrets = []
team1_player3_damage_taken = []
team1_player3_gold_earned = []
team1_player3_champ_level = []
team1_player3_wards_bought = []
team1_player3_vision_score = []
team1_player3_crowd_control = []
team1_player3_role = []
team1_player3_lane = []

team1_player4_champion = []
team1_player4_item0 = []
team1_player4_item1 = []
team1_player4_item2 = []
team1_player4_item3 = []
team1_player4_item4 = []
team1_player4_item5 = []
team1_player4_item6 = []
team1_player4_kills = []
team1_player4_deaths = []
team1_player4_assists = []
team1_player4_killing_spree = []
team1_player4_multikills = []
team1_player4_total_damage = []
team1_player4_damage_to_objectives = []
team1_player4_damage_to_turrets = []
team1_player4_damage_taken = []
team1_player4_gold_earned = []
team1_player4_champ_level = []
team1_player4_wards_bought = []
team1_player4_vision_score = []
team1_player4_crowd_control = []
team1_player4_role = []
team1_player4_lane = []

team1_player5_champion = []
team1_player5_item0 = []
team1_player5_item1 = []
team1_player5_item2 = []
team1_player5_item3 = []
team1_player5_item4 = []
team1_player5_item5 = []
team1_player5_item6 = []
team1_player5_kills = []
team1_player5_deaths = []
team1_player5_assists = []
team1_player5_killing_spree = []
team1_player5_multikills = []
team1_player5_total_damage = []
team1_player5_damage_to_objectives = []
team1_player5_damage_to_turrets = []
team1_player5_damage_taken = []
team1_player5_gold_earned = []
team1_player5_champ_level = []
team1_player5_wards_bought = []
team1_player5_vision_score = []
team1_player5_crowd_control = []
team1_player5_role = []
team1_player5_lane = []

team2_barons = []
team2_dragons = []
team2_ban1 = []
team2_ban2 = []
team2_ban3 = []
team2_ban4 = []
team2_ban5 = []

team2_player1_champion = []
team2_player1_item0 = []
team2_player1_item1 = []
team2_player1_item2 = []
team2_player1_item3 = []
team2_player1_item4 = []
team2_player1_item5 = []
team2_player1_item6 = []
team2_player1_kills = []
team2_player1_deaths = []
team2_player1_assists = []
team2_player1_killing_spree = []
team2_player1_multikills = []
team2_player1_total_damage = []
team2_player1_damage_to_objectives = []
team2_player1_damage_to_turrets = []
team2_player1_damage_taken = []
team2_player1_gold_earned = []
team2_player1_champ_level = []
team2_player1_wards_bought = []
team2_player1_vision_score = []
team2_player1_crowd_control = []
team2_player1_role = []
team2_player1_lane = []

team2_player2_champion = []
team2_player2_item0 = []
team2_player2_item1 = []
team2_player2_item2 = []
team2_player2_item3 = []
team2_player2_item4 = []
team2_player2_item5 = []
team2_player2_item6 = []
team2_player2_kills = []
team2_player2_deaths = []
team2_player2_assists = []
team2_player2_killing_spree = []
team2_player2_multikills = []
team2_player2_total_damage = []
team2_player2_damage_to_objectives = []
team2_player2_damage_to_turrets = []
team2_player2_damage_taken = []
team2_player2_gold_earned = []
team2_player2_champ_level = []
team2_player2_wards_bought = []
team2_player2_vision_score = []
team2_player2_crowd_control = []
team2_player2_role = []
team2_player2_lane = []

team2_player3_champion = []
team2_player3_item0 = []
team2_player3_item1 = []
team2_player3_item2 = []
team2_player3_item3 = []
team2_player3_item4 = []
team2_player3_item5 = []
team2_player3_item6 = []
team2_player3_kills = []
team2_player3_deaths = []
team2_player3_assists = []
team2_player3_killing_spree = []
team2_player3_multikills = []
team2_player3_total_damage = []
team2_player3_damage_to_objectives = []
team2_player3_damage_to_turrets = []
team2_player3_damage_taken = []
team2_player3_gold_earned = []
team2_player3_champ_level = []
team2_player3_wards_bought = []
team2_player3_vision_score = []
team2_player3_crowd_control = []
team2_player3_role = []
team2_player3_lane = []

team2_player4_champion = []
team2_player4_item0 = []
team2_player4_item1 = []
team2_player4_item2 = []
team2_player4_item3 = []
team2_player4_item4 = []
team2_player4_item5 = []
team2_player4_item6 = []
team2_player4_kills = []
team2_player4_deaths = []
team2_player4_assists = []
team2_player4_killing_spree = []
team2_player4_multikills = []
team2_player4_total_damage = []
team2_player4_damage_to_objectives = []
team2_player4_damage_to_turrets = []
team2_player4_damage_taken = []
team2_player4_gold_earned = []
team2_player4_champ_level = []
team2_player4_wards_bought = []
team2_player4_vision_score = []
team2_player4_crowd_control = []
team2_player4_role = []
team2_player4_lane = []

team2_player5_champion = []
team2_player5_item0 = []
team2_player5_item1 = []
team2_player5_item2 = []
team2_player5_item3 = []
team2_player5_item4 = []
team2_player5_item5 = []
team2_player5_item6 = []
team2_player5_kills = []
team2_player5_deaths = []
team2_player5_assists = []
team2_player5_killing_spree = []
team2_player5_multikills = []
team2_player5_total_damage = []
team2_player5_damage_to_objectives = []
team2_player5_damage_to_turrets = []
team2_player5_damage_taken = []
team2_player5_gold_earned = []
team2_player5_champ_level = []
team2_player5_wards_bought = []
team2_player5_vision_score = []
team2_player5_crowd_control = []
team2_player5_role = []
team2_player5_lane = []

In [39]:
mydict[299]['teams'][0]

{'teamId': 100,
 'win': 'Win',
 'firstBlood': False,
 'firstTower': False,
 'firstInhibitor': True,
 'firstBaron': True,
 'firstDragon': True,
 'firstRiftHerald': False,
 'towerKills': 5,
 'inhibitorKills': 1,
 'baronKills': 1,
 'dragonKills': 2,
 'vilemawKills': 0,
 'riftHeraldKills': 0,
 'dominionVictoryScore': 0,
 'bans': []}

In [46]:
for game in mydict:
    try:
        # Bans and count of dragons and barons
        team1_ban1.append(game['teams'][0]['bans'][0]['championId'])
        team1_ban2.append(game['teams'][0]['bans'][1]['championId'])
        team1_ban3.append(game['teams'][0]['bans'][2]['championId'])
        team1_ban4.append(game['teams'][0]['bans'][3]['championId'])
        team1_ban5.append(game['teams'][0]['bans'][4]['championId'])
        team1_barons.append(game['teams'][0]['baronKills'])
        team1_dragons.append(game['teams'][0]['dragonKills'])
        
        # General game info
        gameId.append(game['gameId'])
        duration.append(game['gameDuration'])

        # Champions picked in game
        team1_player1_champion.append(game['participants'][0]['championId'])
        team1_player2_champion.append(game['participants'][1]['championId'])
        team1_player3_champion.append(game['participants'][2]['championId'])
        team1_player4_champion.append(game['participants'][3]['championId'])
        team1_player5_champion.append(game['participants'][4]['championId'])
        team2_player1_champion.append(game['participants'][5]['championId'])
        team2_player2_champion.append(game['participants'][6]['championId'])
        team2_player3_champion.append(game['participants'][7]['championId'])
        team2_player4_champion.append(game['participants'][8]['championId'])
        team2_player5_champion.append(game['participants'][9]['championId'])

        # Items bought by each player
        team1_player1_item0.append(game['participants'][0]['stats']['item0'])
        team1_player1_item1.append(game['participants'][0]['stats']['item1'])
        team1_player1_item2.append(game['participants'][0]['stats']['item2'])
        team1_player1_item3.append(game['participants'][0]['stats']['item3'])
        team1_player1_item4.append(game['participants'][0]['stats']['item4'])
        team1_player1_item5.append(game['participants'][0]['stats']['item5'])
        team1_player1_item6.append(game['participants'][0]['stats']['item6'])

        team1_player2_item0.append(game['participants'][1]['stats']['item0'])
        team1_player2_item1.append(game['participants'][1]['stats']['item1'])
        team1_player2_item2.append(game['participants'][1]['stats']['item2'])
        team1_player2_item3.append(game['participants'][1]['stats']['item3'])
        team1_player2_item4.append(game['participants'][1]['stats']['item4'])
        team1_player2_item5.append(game['participants'][1]['stats']['item5'])
        team1_player2_item6.append(game['participants'][1]['stats']['item6'])

        team1_player3_item0.append(game['participants'][2]['stats']['item0'])
        team1_player3_item1.append(game['participants'][2]['stats']['item1'])
        team1_player3_item2.append(game['participants'][2]['stats']['item2'])
        team1_player3_item3.append(game['participants'][2]['stats']['item3'])
        team1_player3_item4.append(game['participants'][2]['stats']['item4'])
        team1_player3_item5.append(game['participants'][2]['stats']['item5'])
        team1_player3_item6.append(game['participants'][2]['stats']['item6'])

        team1_player4_item0.append(game['participants'][3]['stats']['item0'])
        team1_player4_item1.append(game['participants'][3]['stats']['item1'])
        team1_player4_item2.append(game['participants'][3]['stats']['item2'])
        team1_player4_item3.append(game['participants'][3]['stats']['item3'])
        team1_player4_item4.append(game['participants'][3]['stats']['item4'])
        team1_player4_item5.append(game['participants'][3]['stats']['item5'])
        team1_player4_item6.append(game['participants'][3]['stats']['item6'])

        team1_player5_item0.append(game['participants'][4]['stats']['item0'])
        team1_player5_item1.append(game['participants'][4]['stats']['item1'])
        team1_player5_item2.append(game['participants'][4]['stats']['item2'])
        team1_player5_item3.append(game['participants'][4]['stats']['item3'])
        team1_player5_item4.append(game['participants'][4]['stats']['item4'])
        team1_player5_item5.append(game['participants'][4]['stats']['item5'])
        team1_player5_item6.append(game['participants'][4]['stats']['item6'])

        team2_player1_item0.append(game['participants'][5]['stats']['item0'])
        team2_player1_item1.append(game['participants'][5]['stats']['item1'])
        team2_player1_item2.append(game['participants'][5]['stats']['item2'])
        team2_player1_item3.append(game['participants'][5]['stats']['item3'])
        team2_player1_item4.append(game['participants'][5]['stats']['item4'])
        team2_player1_item5.append(game['participants'][5]['stats']['item5'])
        team2_player1_item6.append(game['participants'][5]['stats']['item6'])

        team2_player2_item0.append(game['participants'][6]['stats']['item0'])
        team2_player2_item1.append(game['participants'][6]['stats']['item1'])
        team2_player2_item2.append(game['participants'][6]['stats']['item2'])
        team2_player2_item3.append(game['participants'][6]['stats']['item3'])
        team2_player2_item4.append(game['participants'][6]['stats']['item4'])
        team2_player2_item5.append(game['participants'][6]['stats']['item5'])
        team2_player2_item6.append(game['participants'][6]['stats']['item6'])

        team2_player3_item0.append(game['participants'][7]['stats']['item0'])
        team2_player3_item1.append(game['participants'][7]['stats']['item1'])
        team2_player3_item2.append(game['participants'][7]['stats']['item2'])
        team2_player3_item3.append(game['participants'][7]['stats']['item3'])
        team2_player3_item4.append(game['participants'][7]['stats']['item4'])
        team2_player3_item5.append(game['participants'][7]['stats']['item5'])
        team2_player3_item6.append(game['participants'][7]['stats']['item6'])

        team2_player4_item0.append(game['participants'][8]['stats']['item0'])
        team2_player4_item1.append(game['participants'][8]['stats']['item1'])
        team2_player4_item2.append(game['participants'][8]['stats']['item2'])
        team2_player4_item3.append(game['participants'][8]['stats']['item3'])
        team2_player4_item4.append(game['participants'][8]['stats']['item4'])
        team2_player4_item5.append(game['participants'][8]['stats']['item5'])
        team2_player4_item6.append(game['participants'][8]['stats']['item6'])

        team2_player5_item0.append(game['participants'][9]['stats']['item0'])
        team2_player5_item1.append(game['participants'][9]['stats']['item1'])
        team2_player5_item2.append(game['participants'][9]['stats']['item2'])
        team2_player5_item3.append(game['participants'][9]['stats']['item3'])
        team2_player5_item4.append(game['participants'][9]['stats']['item4'])
        team2_player5_item5.append(game['participants'][9]['stats']['item5'])
        team2_player5_item6.append(game['participants'][9]['stats']['item6'])

        # Number of kills by each player
        team1_player1_kills.append(game['participants'][0]['stats']['kills'])
        team1_player2_kills.append(game['participants'][1]['stats']['kills'])
        team1_player3_kills.append(game['participants'][2]['stats']['kills'])
        team1_player4_kills.append(game['participants'][3]['stats']['kills'])
        team1_player5_kills.append(game['participants'][4]['stats']['kills'])
        team2_player1_kills.append(game['participants'][5]['stats']['kills'])
        team2_player2_kills.append(game['participants'][6]['stats']['kills'])
        team2_player3_kills.append(game['participants'][7]['stats']['kills'])
        team2_player4_kills.append(game['participants'][8]['stats']['kills'])
        team2_player5_kills.append(game['participants'][9]['stats']['kills'])

        # Number of deaths by each player
        team1_player1_deaths.append(game['participants'][0]['stats']['deaths'])
        team1_player2_deaths.append(game['participants'][1]['stats']['deaths'])
        team1_player3_deaths.append(game['participants'][2]['stats']['deaths'])
        team1_player4_deaths.append(game['participants'][3]['stats']['deaths'])
        team1_player5_deaths.append(game['participants'][4]['stats']['deaths'])
        team2_player1_deaths.append(game['participants'][5]['stats']['deaths'])
        team2_player2_deaths.append(game['participants'][6]['stats']['deaths'])
        team2_player3_deaths.append(game['participants'][7]['stats']['deaths'])
        team2_player4_deaths.append(game['participants'][8]['stats']['deaths'])
        team2_player5_deaths.append(game['participants'][9]['stats']['deaths'])

        # Number of assists by each player
        team1_player1_assists.append(game['participants'][0]['stats']['assists'])
        team1_player2_assists.append(game['participants'][1]['stats']['assists'])
        team1_player3_assists.append(game['participants'][2]['stats']['assists'])
        team1_player4_assists.append(game['participants'][3]['stats']['assists'])
        team1_player5_assists.append(game['participants'][4]['stats']['assists'])
        team2_player1_assists.append(game['participants'][5]['stats']['assists'])
        team2_player2_assists.append(game['participants'][6]['stats']['assists'])
        team2_player3_assists.append(game['participants'][7]['stats']['assists'])
        team2_player4_assists.append(game['participants'][8]['stats']['assists'])
        team2_player5_assists.append(game['participants'][9]['stats']['assists'])

        # Largest killing spree by each player
        team1_player1_killing_spree.append(game['participants'][0]['stats']['largestKillingSpree'])
        team1_player2_killing_spree.append(game['participants'][1]['stats']['largestKillingSpree'])
        team1_player3_killing_spree.append(game['participants'][2]['stats']['largestKillingSpree'])
        team1_player4_killing_spree.append(game['participants'][3]['stats']['largestKillingSpree'])
        team1_player5_killing_spree.append(game['participants'][4]['stats']['largestKillingSpree'])
        team2_player1_killing_spree.append(game['participants'][5]['stats']['largestKillingSpree'])
        team2_player2_killing_spree.append(game['participants'][6]['stats']['largestKillingSpree'])
        team2_player3_killing_spree.append(game['participants'][7]['stats']['largestKillingSpree'])
        team2_player4_killing_spree.append(game['participants'][8]['stats']['largestKillingSpree'])
        team2_player5_killing_spree.append(game['participants'][9]['stats']['largestKillingSpree'])

        # Largest multikill by each player
        team1_player1_multikills.append(game['participants'][0]['stats']['largestMultiKill'])
        team1_player2_multikills.append(game['participants'][1]['stats']['largestMultiKill'])
        team1_player3_multikills.append(game['participants'][2]['stats']['largestMultiKill'])
        team1_player4_multikills.append(game['participants'][3]['stats']['largestMultiKill'])
        team1_player5_multikills.append(game['participants'][4]['stats']['largestMultiKill'])
        team2_player1_multikills.append(game['participants'][5]['stats']['largestMultiKill'])
        team2_player2_multikills.append(game['participants'][6]['stats']['largestMultiKill'])
        team2_player3_multikills.append(game['participants'][7]['stats']['largestMultiKill'])
        team2_player4_multikills.append(game['participants'][8]['stats']['largestMultiKill'])
        team2_player5_multikills.append(game['participants'][9]['stats']['largestMultiKill'])

        # Total damage done to other players
        team1_player1_total_damage.append(game['participants'][0]['stats']['totalDamageDealtToChampions'])
        team1_player2_total_damage.append(game['participants'][1]['stats']['totalDamageDealtToChampions'])
        team1_player3_total_damage.append(game['participants'][2]['stats']['totalDamageDealtToChampions'])
        team1_player4_total_damage.append(game['participants'][3]['stats']['totalDamageDealtToChampions'])
        team1_player5_total_damage.append(game['participants'][4]['stats']['totalDamageDealtToChampions'])
        team2_player1_total_damage.append(game['participants'][5]['stats']['totalDamageDealtToChampions'])
        team2_player2_total_damage.append(game['participants'][6]['stats']['totalDamageDealtToChampions'])
        team2_player3_total_damage.append(game['participants'][7]['stats']['totalDamageDealtToChampions'])
        team2_player4_total_damage.append(game['participants'][8]['stats']['totalDamageDealtToChampions'])
        team2_player5_total_damage.append(game['participants'][9]['stats']['totalDamageDealtToChampions'])

        # Total damage done to objectives
        team1_player1_damage_to_objectives.append(game['participants'][0]['stats']['damageDealtToObjectives'])
        team1_player2_damage_to_objectives.append(game['participants'][1]['stats']['damageDealtToObjectives'])
        team1_player3_damage_to_objectives.append(game['participants'][2]['stats']['damageDealtToObjectives'])
        team1_player4_damage_to_objectives.append(game['participants'][3]['stats']['damageDealtToObjectives'])
        team1_player5_damage_to_objectives.append(game['participants'][4]['stats']['damageDealtToObjectives'])
        team2_player1_damage_to_objectives.append(game['participants'][5]['stats']['damageDealtToObjectives'])
        team2_player2_damage_to_objectives.append(game['participants'][6]['stats']['damageDealtToObjectives'])
        team2_player3_damage_to_objectives.append(game['participants'][7]['stats']['damageDealtToObjectives'])
        team2_player4_damage_to_objectives.append(game['participants'][8]['stats']['damageDealtToObjectives'])
        team2_player5_damage_to_objectives.append(game['participants'][9]['stats']['damageDealtToObjectives'])

        # Total damage done to turrets
        team1_player1_damage_to_turrets.append(game['participants'][0]['stats']['damageDealtToTurrets'])
        team1_player2_damage_to_turrets.append(game['participants'][1]['stats']['damageDealtToTurrets'])
        team1_player3_damage_to_turrets.append(game['participants'][2]['stats']['damageDealtToTurrets'])
        team1_player4_damage_to_turrets.append(game['participants'][3]['stats']['damageDealtToTurrets'])
        team1_player5_damage_to_turrets.append(game['participants'][4]['stats']['damageDealtToTurrets'])
        team2_player1_damage_to_turrets.append(game['participants'][5]['stats']['damageDealtToTurrets'])
        team2_player2_damage_to_turrets.append(game['participants'][6]['stats']['damageDealtToTurrets'])
        team2_player3_damage_to_turrets.append(game['participants'][7]['stats']['damageDealtToTurrets'])
        team2_player4_damage_to_turrets.append(game['participants'][8]['stats']['damageDealtToTurrets'])
        team2_player5_damage_to_turrets.append(game['participants'][9]['stats']['damageDealtToTurrets'])

        # Total Damage Taken by each player
        team1_player1_damage_taken.append(game['participants'][0]['stats']['totalDamageTaken'])
        team1_player2_damage_taken.append(game['participants'][1]['stats']['totalDamageTaken'])
        team1_player3_damage_taken.append(game['participants'][2]['stats']['totalDamageTaken'])
        team1_player4_damage_taken.append(game['participants'][3]['stats']['totalDamageTaken'])
        team1_player5_damage_taken.append(game['participants'][4]['stats']['totalDamageTaken'])
        team2_player1_damage_taken.append(game['participants'][5]['stats']['totalDamageTaken'])
        team2_player2_damage_taken.append(game['participants'][6]['stats']['totalDamageTaken'])
        team2_player3_damage_taken.append(game['participants'][7]['stats']['totalDamageTaken'])
        team2_player4_damage_taken.append(game['participants'][8]['stats']['totalDamageTaken'])
        team2_player5_damage_taken.append(game['participants'][9]['stats']['totalDamageTaken'])

        # Total Gold Earned by each player
        team1_player1_gold_earned.append(game['participants'][0]['stats']['goldEarned'])
        team1_player2_gold_earned.append(game['participants'][1]['stats']['goldEarned'])
        team1_player3_gold_earned.append(game['participants'][2]['stats']['goldEarned'])
        team1_player4_gold_earned.append(game['participants'][3]['stats']['goldEarned'])
        team1_player5_gold_earned.append(game['participants'][4]['stats']['goldEarned'])
        team2_player1_gold_earned.append(game['participants'][5]['stats']['goldEarned'])
        team2_player2_gold_earned.append(game['participants'][6]['stats']['goldEarned'])
        team2_player3_gold_earned.append(game['participants'][7]['stats']['goldEarned'])
        team2_player4_gold_earned.append(game['participants'][8]['stats']['goldEarned'])
        team2_player5_gold_earned.append(game['participants'][9]['stats']['goldEarned'])

        # Champion level at the end of the game
        team1_player1_champ_level.append(game['participants'][0]['stats']['champLevel'])
        team1_player2_champ_level.append(game['participants'][1]['stats']['champLevel'])
        team1_player3_champ_level.append(game['participants'][2]['stats']['champLevel'])
        team1_player4_champ_level.append(game['participants'][3]['stats']['champLevel'])
        team1_player5_champ_level.append(game['participants'][4]['stats']['champLevel'])
        team2_player1_champ_level.append(game['participants'][5]['stats']['champLevel'])
        team2_player2_champ_level.append(game['participants'][6]['stats']['champLevel'])
        team2_player3_champ_level.append(game['participants'][7]['stats']['champLevel'])
        team2_player4_champ_level.append(game['participants'][8]['stats']['champLevel'])
        team2_player5_champ_level.append(game['participants'][9]['stats']['champLevel'])

        # Number of wards bought by each player
        team1_player1_wards_bought.append(game['participants'][0]['stats']['visionWardsBoughtInGame'])
        team1_player2_wards_bought.append(game['participants'][1]['stats']['visionWardsBoughtInGame'])
        team1_player3_wards_bought.append(game['participants'][2]['stats']['visionWardsBoughtInGame'])
        team1_player4_wards_bought.append(game['participants'][3]['stats']['visionWardsBoughtInGame'])
        team1_player5_wards_bought.append(game['participants'][4]['stats']['visionWardsBoughtInGame'])
        team2_player1_wards_bought.append(game['participants'][5]['stats']['visionWardsBoughtInGame'])
        team2_player2_wards_bought.append(game['participants'][6]['stats']['visionWardsBoughtInGame'])
        team2_player3_wards_bought.append(game['participants'][7]['stats']['visionWardsBoughtInGame'])
        team2_player4_wards_bought.append(game['participants'][8]['stats']['visionWardsBoughtInGame'])
        team2_player5_wards_bought.append(game['participants'][9]['stats']['visionWardsBoughtInGame'])

        # Vision Score of each player
        team1_player1_vision_score.append(game['participants'][0]['stats']['visionScore'])
        team1_player2_vision_score.append(game['participants'][1]['stats']['visionScore'])
        team1_player3_vision_score.append(game['participants'][2]['stats']['visionScore'])
        team1_player4_vision_score.append(game['participants'][3]['stats']['visionScore'])
        team1_player5_vision_score.append(game['participants'][4]['stats']['visionScore'])
        team2_player1_vision_score.append(game['participants'][5]['stats']['visionScore'])
        team2_player2_vision_score.append(game['participants'][6]['stats']['visionScore'])
        team2_player3_vision_score.append(game['participants'][7]['stats']['visionScore'])
        team2_player4_vision_score.append(game['participants'][8]['stats']['visionScore'])
        team2_player5_vision_score.append(game['participants'][9]['stats']['visionScore'])

        # Crowd Control by each player
        team1_player1_crowd_control.append(game['participants'][0]['stats']['timeCCingOthers'])
        team1_player2_crowd_control.append(game['participants'][1]['stats']['timeCCingOthers'])
        team1_player3_crowd_control.append(game['participants'][2]['stats']['timeCCingOthers'])
        team1_player4_crowd_control.append(game['participants'][3]['stats']['timeCCingOthers'])
        team1_player5_crowd_control.append(game['participants'][4]['stats']['timeCCingOthers'])
        team2_player1_crowd_control.append(game['participants'][5]['stats']['timeCCingOthers'])
        team2_player2_crowd_control.append(game['participants'][6]['stats']['timeCCingOthers'])
        team2_player3_crowd_control.append(game['participants'][7]['stats']['timeCCingOthers'])
        team2_player4_crowd_control.append(game['participants'][8]['stats']['timeCCingOthers'])
        team2_player5_crowd_control.append(game['participants'][9]['stats']['timeCCingOthers'])

        # Role of each player
        team1_player1_role.append(game['participants'][0]['timeline']['role'])
        team1_player2_role.append(game['participants'][1]['timeline']['role'])
        team1_player3_role.append(game['participants'][2]['timeline']['role'])
        team1_player4_role.append(game['participants'][3]['timeline']['role'])
        team1_player5_role.append(game['participants'][4]['timeline']['role'])
        team2_player1_role.append(game['participants'][5]['timeline']['role'])
        team2_player2_role.append(game['participants'][6]['timeline']['role'])
        team2_player3_role.append(game['participants'][7]['timeline']['role'])
        team2_player4_role.append(game['participants'][8]['timeline']['role'])
        team2_player5_role.append(game['participants'][9]['timeline']['role'])

        # Lane of each player
        team1_player1_lane.append(game['participants'][0]['timeline']['lane'])
        team1_player2_lane.append(game['participants'][1]['timeline']['lane'])
        team1_player3_lane.append(game['participants'][2]['timeline']['lane'])
        team1_player4_lane.append(game['participants'][3]['timeline']['lane'])
        team1_player5_lane.append(game['participants'][4]['timeline']['lane'])
        team2_player1_lane.append(game['participants'][5]['timeline']['lane'])
        team2_player2_lane.append(game['participants'][6]['timeline']['lane'])
        team2_player3_lane.append(game['participants'][7]['timeline']['lane'])
        team2_player4_lane.append(game['participants'][8]['timeline']['lane'])
        team2_player5_lane.append(game['participants'][9]['timeline']['lane'])

        # Bans and count of dragons and barons
        team2_barons.append(game['teams'][1]['baronKills'])
        team2_dragons.append(game['teams'][1]['dragonKills'])
        team2_ban1.append(game['teams'][1]['bans'][0]['championId'])
        team2_ban2.append(game['teams'][1]['bans'][1]['championId'])
        team2_ban3.append(game['teams'][1]['bans'][2]['championId'])
        team2_ban4.append(game['teams'][1]['bans'][3]['championId'])
        team2_ban5.append(game['teams'][1]['bans'][4]['championId'])

    except:
        pass

In [47]:
match_details = pd.DataFrame()

team1_barons = []
team1_dragons = []
team1_ban1 = []
team1_ban2 = []
team1_ban3 = []
team1_ban4 = []
team1_ban5 = []

team1_player1_champion = []
team1_player1_item0 = []
team1_player1_item1 = []
team1_player1_item2 = []
team1_player1_item3 = []
team1_player1_item4 = []
team1_player1_item5 = []
team1_player1_item6 = []
team1_player1_kills = []
team1_player1_deaths = []
team1_player1_assists = []
team1_player1_killing_spree = []
team1_player1_multikills = []
team1_player1_total_damage = []
team1_player1_damage_to_objectives = []
team1_player1_damage_to_turrets = []
team1_player1_damage_taken = []
team1_player1_gold_earned = []
team1_player1_champ_level = []
team1_player1_wards_bought = []
team1_player1_vision_score = []
team1_player1_crowd_control = []
team1_player1_role = []
team1_player1_lane = []

In [48]:
match_details['GameId'] = gameId
match_details['Duration'] = duration

match_details['Team1_Barons'] = team1_barons
match_details['Team1_Dragons'] = team1_dragons
match_details['Team1_Ban1'] = team1_ban1
match_details['Team1_Ban2'] = team1_ban2
match_details['Team1_Ban3'] = team1_ban3
match_details['Team1_Ban4'] = team1_ban4
match_details['Team1_Ban5'] = team1_ban5

match_details['Team2_Barons'] = team2_barons
match_details['Team2_Dragons'] = team2_dragons
match_details['Team2_Ban1'] = team2_ban1
match_details['Team2_Ban2'] = team2_ban2
match_details['Team2_Ban3'] = team2_ban3
match_details['Team2_Ban4'] = team2_ban4
match_details['Team2_Ban5'] = team2_ban5



ValueError: Length of values does not match length of index